In [1]:
import gc
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import time

import category_encoders as ce
import catboost
from sklearn import metrics

import sys
sys.path.append("../utils")
from metrics import compute_recall_at4, compute_normalized_gini, compute_amex_metric
#from messaging import send_message

pd.set_option('display.max_columns', None)

In [2]:
# metrics in catboost format
class AmexMetric:
    
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):    
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        scores = approxes[0]
        target = target
        return compute_amex_metric(target, scores), 1.

    def get_final_error(self, error, weight):
        return error

class RecallAt4:
    
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        scores = np.array(approxes[0])
        target = np.array(target)                                   
        return compute_recall_at4(target, scores), 1.

    def get_final_error(self, error, weight):
        return error

class NormGini:
    
    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        scores = np.array(approxes[0])
        target = np.array(target)                   
        return compute_normalized_gini(target, scores), 1.

    def get_final_error(self, error, weight):
        return error

In [3]:
# CONFIG PARAMS
N_REPEATS = 5
DATASET_VERSION = "08"

In [4]:
OOF_PATH = Path(f"../data/oof/catboost-dsv{DATASET_VERSION}")
SUB_PATH = Path(f"../data/subs/catboost-dsv{DATASET_VERSION}")
ART_PATH = Path(f"../artifacts/catboost-dsv{DATASET_VERSION}")

if not OOF_PATH.exists():
    OOF_PATH.mkdir(parents=True, exist_ok=True)
if not SUB_PATH.exists():
    SUB_PATH.mkdir(parents=True, exist_ok=True)
if not ART_PATH.exists():
    ART_PATH.mkdir(parents=True, exist_ok=True)

***
## load and prepare data

In [5]:
train = pd.read_parquet(f"../data/processed/dsv{DATASET_VERSION}/train.parquet")
train_labels = pd.read_csv("../data/raw/train_labels.csv", index_col="customer_ID")

In [6]:
input_feats = train.columns.tolist()
len(input_feats)

1149

In [7]:
train = pd.merge(train, train_labels, how="inner", left_index=True, right_index=True)
del train_labels
gc.collect()

0

***
## model training

train with repeated cross validation

In [8]:
model_params = {
    'eval_metric':AmexMetric(),
    'learning_rate': 0.05,
    'nan_mode':'Min',
    'random_seed': 2112,
    'auto_class_weights': None,
    'bootstrap_type': 'Bernoulli',
    'depth': 6,
    'rsm': 0.1,
    'iterations': 5000,
    'l2_leaf_reg': 37.05327214190692,
    'min_data_in_leaf': 600,
    'random_strength': 27.64341860474617,
    'subsample': 0.9,
    # early stopping
    'early_stopping_rounds':5000,
    'use_best_model': False,
}

In [9]:
def train_models(dataframe: pd.DataFrame, n_folds: int = 5,) -> tuple:
    
    models = list()
    
    # dataframe to store the oof predictions
    oof = dataframe[["target"]].copy()
    oof["pred"] = -1

    for fold in range(n_folds):
        
        seed = repetition*5 + fold
        _model_params = dict(model_params)
        _model_params["random_seed"] = seed
        
        print(f" training model {fold+1}/{n_folds} ".center(100, "#"))
        print("seed:", seed)
        
        train_df = dataframe.query("fold != @fold").copy()
        valid_df = dataframe.query("fold == @fold").copy()
                
        train_dset = catboost.Pool(
            data=train_df.loc[:,input_feats],
            label=train_df.loc[:,"target"].values,
        )
        valid_dset = catboost.Pool(
            data=valid_df.loc[:,input_feats],
            label=valid_df.loc[:,"target"].values,
        )
        
        model = catboost.CatBoostClassifier(**_model_params)
        model.fit(
            train_dset,
            eval_set=valid_dset,
            verbose=50,
        )     
        
        oof.loc[valid_df.index,"pred"] = model.predict(valid_dset, prediction_type="Probability")[:,1]
        
        models.append(model)
        del train_df,valid_df,train_dset,valid_dset
        gc.collect()
    
    return models,oof

In [10]:
# implement repeated cross validation
sorted(glob("../data/processed/cv*.csv"))

['../data/processed/cv0.csv',
 '../data/processed/cv1.csv',
 '../data/processed/cv2.csv',
 '../data/processed/cv3.csv',
 '../data/processed/cv4.csv',
 '../data/processed/cv5.csv',
 '../data/processed/cv6.csv',
 '../data/processed/cv7.csv',
 '../data/processed/cv8.csv',
 '../data/processed/cv9.csv']

In [11]:
%%time 

all_models = list()
all_oof_dfs = list()

for repetition in range(N_REPEATS):
    print(f" repeated cross-validation step: {repetition+1}/{N_REPEATS} ".center(100, "#"))

    folds = pd.read_csv(f'../data/processed/cv{repetition}.csv', index_col="customer_ID")
    _train = pd.merge(train, folds, how="inner", left_index=True, right_index=True).reset_index(drop=True)
    
    tic = time.time()
    models,oof = train_models(_train)
    tac = time.time()
    print(f"Training time: {(tac-tic)/60} min.")
          
    # oof metrics
    print("OOF recall_at4:", compute_recall_at4(oof.target.values, oof.pred.values))
    print("OOF normalized_gini:", compute_normalized_gini(oof.target.values, oof.pred.values))
    print("OOF competition metric:", compute_amex_metric(oof.target.values, oof.pred.values))
    
    all_models.append(models)
    all_oof_dfs.append(oof)
    
    # save oof predictions
    oof.to_csv(OOF_PATH/f"oof-cv{repetition}.csv", index=False)
    # save models
    for fold,_model in enumerate(models):
        _model.save_model(ART_PATH/f"/model-cv{repetition}-fold{fold}.cbm", format="cbm")
          
    del _train, folds; gc.collect()

############################### repeated cross-validation step: 1/5 ################################


######################################## training model 1/5 ########################################
seed: 0


0:	learn: 0.4684901	test: 0.4679450	best: 0.4679450 (0)	total: 2.8s	remaining: 3h 53m 8s


50:	learn: 0.7404698	test: 0.7451579	best: 0.7451579 (50)	total: 8.93s	remaining: 14m 26s


100:	learn: 0.7538769	test: 0.7578206	best: 0.7578206 (100)	total: 15.1s	remaining: 12m 14s


...

4900:	learn: 0.8899311	test: 0.7918097	best: 0.7925739 (4327)	total: 10m 19s	remaining: 12.5s


4950:	learn: 0.8905408	test: 0.7915649	best: 0.7925739 (4327)	total: 10m 25s	remaining: 6.19s


4999:	learn: 0.8911514	test: 0.7915528	best: 0.7925739 (4327)	total: 10m 31s	remaining: 0us

bestTest = 0.7925739473
bestIteration = 4327



Training time: 53.038133068879446 min.
OOF recall_at4: 0.6651967549735752
OOF normalized_gini: 0.9242181302074143
OOF competition metric: 0.7947074425904948


CPU times: user 2d 6h 26min 1s, sys: 2h 58min 57s, total: 2d 9h 24min 58s
Wall time: 4h 24min 46s


In [12]:
results = list()

for oof in all_oof_dfs:  
    r = {
        "recall_at4": compute_recall_at4(oof.target.values, oof.pred.values),
        "gini": compute_normalized_gini(oof.target.values, oof.pred.values),
        "metric": compute_amex_metric(oof.target.values, oof.pred.values),
    }
    results.append(r)

results = pd.DataFrame(results)
display(results)

print("\nmean:")
display(results.mean(axis=0))

print("\nstd:")
display(results.std(axis=0))

,recall_at4,gini,metric
0,0.665003,0.924135,0.794569
1,0.665003,0.924151,0.794577
2,0.665365,0.924269,0.794817
3,0.664953,0.924230,0.794592
4,0.665197,0.924218,0.794707



mean:


recall_at4    0.665104
gini          0.924201
metric        0.794652
dtype: float64


std:


recall_at4    0.000173
gini          0.000056
metric        0.000108
dtype: float64

In [13]:
del train
gc.collect()

21

***
## make predictions and submit

In [14]:
def make_predictions(dataframe:pd.DataFrame, input_feats:list, models:list) -> np.array:
    preds = [
        model.predict(dataframe[input_feats], prediction_type="Probability")[:,1] 
        for model in models
    ]
    return np.mean(preds, axis=0)    

In [15]:
test = pd.read_parquet(f"../data/processed/dsv{DATASET_VERSION}/test.parquet")
sub = pd.read_csv("../data/raw/sample_submission.csv")

In [16]:
%%time

all_preds = list()

for repetition in range(N_REPEATS):
    if "prediction" in sub.columns:
        sub.drop("prediction", axis=1, inplace=True)
    if "prediction" in test.columns:
        test.drop("prediction", axis=1, inplace=True)
        
    models = all_models[repetition]
    preds = make_predictions(test, input_feats, models)
    all_preds.append(preds)
       
    test["prediction"] = preds
    sub["prediction"] = test.loc[sub.customer_ID.values,"prediction"].values
    assert sub.prediction.isna().sum() == 0
    sub.to_csv(SUB_PATH/f"submission-cv{repetition}.csv", index=False)

CPU times: user 11min 12s, sys: 22.8 s, total: 11min 35s
Wall time: 1min 18s


In [17]:
%%time
# predict using all the trained models
if "prediction" in sub.columns:
    sub.drop("prediction", axis=1, inplace=True)
if "prediction" in test.columns:
    test.drop("prediction", axis=1, inplace=True)

test["prediction"] = np.mean(all_preds, axis=0)
sub["prediction"] = test.loc[sub.customer_ID.values,"prediction"].values
assert sub.prediction.isna().sum() == 0
sub.to_csv(SUB_PATH/f"submission-all.csv", index=False)

CPU times: user 3.9 s, sys: 696 ms, total: 4.6 s
Wall time: 4.59 s


***